In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import transforms
import torchvision
import torch.optim as optim

In [10]:
# pretrained = torch.hub.load('pytorch/vision', 'resnet34', pretreined=True)
import torchvision.models as models
import urllib

resnet18 = models.resnet18(pretrained=True, progress=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/telaust/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [13]:

num_classes = 2

resnet18.fc = nn.Sequential(nn.Linear(resnet18.fc.in_features,512),
                                  nn.ReLU(),
                                  nn.Dropout(),
                                  nn.Linear(512, num_classes))

In [44]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize( (0.5, ), (0.5, ) )])

In [45]:
# define datasets
train_data = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

In [46]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=False, num_workers=2)

In [47]:
classes = tuple(train_data.classes)

In [63]:


class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(400, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # 1 layer
        x = self.pool(F.relu(self.conv2(x))) # 2 layer
        
        x = x.view(-1, 16*5*5) # flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
#         self.dropout1 = nn.Dropout2d(0.25)
#         self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
#         x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
#         x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [67]:
net = Net()

In [70]:
crit = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [71]:
for ep in range(3):
    
    for batch_idx, data in enumerate(train_loader, 0):
        images, labels = data
        # zero all gradients
        opt.zero_grad()
        
        # forward pass
        outputs = net(images)
        loss = crit(outputs, labels)
        
        # backprop
        loss.backward()
        opt.step()
        
        pred = torch.max(outputs, 1)
        
        if batch_idx % 100 == 0 and batch_idx != 0:
            print("iteration #{}, epoch #{} ; loss is {}, accuracy is {}".format(batch_idx, ep+1, loss.item()))

iteration #100, epoch #1 and loss is 0.6415295004844666
iteration #200, epoch #1 and loss is 0.8986220359802246


KeyboardInterrupt: 

'b/untitled.txt'